In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import cluster, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
%matplotlib inline

#Import pytorch:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset, random_split
import torchvision
import torchvision.transforms as transforms

In [2]:
# Load data
df = pd.read_csv("data2.csv")
df.rename(columns={'time sample band g': 'time_g', 'time sample band r': 'time_r',
                    'time sample band i': 'time_i','total flux + noise band g': 'tfnbg',
                    'total flux + noise band r': 'tfnbr', 'total flux + noise band i': 'tfnbi',}, inplace=True)

df = df.loc[:, ['images', 'time_g', 'tfnbg']]

In [3]:
# Splitting the attributes 
X = df.iloc[:, 1:].values
Y = df.iloc[:, 0].values

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,  test_size=0.2, random_state=13 )

In [5]:
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

In [6]:
X_train = torch.tensor(X_train).clone().detach()
X_test = torch.tensor(X_test).clone().detach()

In [17]:
X_train

tensor([[ 1.3683, -0.3883],
        [-0.2460, -0.0408],
        [-0.8725,  0.7178],
        ...,
        [-0.5963, -0.3954],
        [-0.5947,  0.0024],
        [ 1.2371, -0.1294]], dtype=torch.float64)

In [7]:
n_train = 1000

In [8]:
# Hyperparameters
N_epochs = 20
batch_size = 1000
learning_rate = 0.001

In [13]:
# Hyperparameters

input_dim = 2

H = 25
output_dim = 4

# Network Class
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.neuron = nn.Sequential(
            nn.Linear(input_dim, H),
            nn.ReLU(),
            nn.Linear(H, output_dim))
            #nn.Softmax())      # because we want an array composed of 10 proba for each digits.
                                # commented because it's already calculated in the cross-entropy loss. 
        
    def forward(self, x):
        x = self.neuron(x)
        return x
    
torch.manual_seed(0)

# Network creation 
model = Net()

# Display the architecture:
print(model)

Net(
  (neuron): Sequential(
    (0): Linear(in_features=2, out_features=25, bias=True)
    (1): ReLU()
    (2): Linear(in_features=25, out_features=4, bias=True)
  )
)


In [14]:
loss_function = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
train_loader = torch.utils.data.DataLoader(dataset=X_train,
                                          batch_size=batch_size,
                                          shuffle=True)

valid_loader = torch.utils.data.DataLoader(dataset=Y_train,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=Y_test,
                                          batch_size=batch_size,
                                          shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=X_test,
                                          batch_size=batch_size,
                                          shuffle=False)



In [16]:
# Train
train_loss = []
valid_loss = []

for epoch in range(N_epochs):
    
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    for data in train_loader:
        #data = data.type(dtype)
        
        output = model(data)
                
        loss = criterion(output, data)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # record train loss
    train_loss.append(loss.item())

    ######################    
    # validate the model #
    ######################
    model.eval() # prep model for evaluation
    for data in valid_loader:
        #data = data.type(dtype)
        output = model(data)
        loss = criterion(output, data)
        
    # record validation loss
    valid_loss.append(loss.item())    
    
    if (epoch==0 or (epoch+1)%50==0):
        print('epoch [{}/{}], train loss:{:.4f}, validation loss:{:.4f}'.format(epoch + 1, num_epochs, train_loss[-1],valid_loss[-1]))
    
print('Final train loss value: %.4f' % train_loss[-1])

# Plot model performance
epochs = range(num_epochs)
plt.figure()
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, valid_loss, 'r-', label='Validation loss')
plt.title('Training and validation losses')
plt.legend();

RuntimeError: expected scalar type Float but found Double